![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2.2: Combinational Logic

## Motivation
In this section you will see how to use Chisel components to implement combinational logic.
We will demonstrate how the 3 of the basic chisel types: UInt - unsigned integer; SInt - signed integer, and Bool - true or false; may be connected and operated upon.
>Notice how all Chisel variables are declared as Scala vals.
Never use a Scala var for a hardware construct, since the construct may never change once defined; only its value may change.
Wires may be used for parameterized types.

In [ ]:
val path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester, TesterOptionsManager}
import chisel3.util._
import chisel3.Driver.emitVerilog

## Basic Operators

Now that you understand how `Module`s are constructed, let's make some hardware! Take a look at the empty module below.

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
}

We've called our class `MyModule`, and it is of type `Module`. This means it gets mapped to a hardware module in Verilog. Our `MyModule` module has one input and one output. The input is a 4-bit unsigned integer (`UInt`), and so is the output. Let's look at different operations we can perform on data.

In [ ]:
class MyModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val two  = 1 + 1
  println(two)
  val utwo = 1.U + 1.U
  println(utwo)
}
println(emitVerilog(new MyModule).substring(216))

We create two `val`s. The first adds two scala `Int`s, so `println` prints out the integer 2. The second `val` adds two `UInt`s together, so `println` sees this as a hardware node and prints out the type name and pointer (`chisel3.core.UInt@d`). Since neither are connected to the output, Chisel drives the output to zero by default. 

What happens if we add `1.U` with the literal `1`? These types are incompatible, as one the former is a hardware wire of value 1, while the latter is a scala value of 1. So Chisel will give a type mismatch error.

<span style="color:blue">**Example 0:**</span>

In [ ]:
class MyModuleTwo extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })

  val twotwo = 1.U + 1
  println(twotwo)
}
println(emitVerilog(new MyModule).substring(216))

It's important to remember the distinction between types when performing operations.

Other common operations are subtraction and multiplication. These are handled on unsigned integers as expected. Let's  see these in action. We show the Verilog, though there's some underlying Chisel features that obfuscate the simple code we would expect.

<span style="color:blue">**Example 1:**</span>

In [ ]:
class MyOperators extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_add = Output(UInt(4.W))
    val out_sub = Output(UInt(4.W))
    val out_mul = Output(UInt(4.W))
  })

  io.out_add := 1.U + 4.U
  io.out_sub := 2.U - 1.U
  io.out_mul := 4.U * 2.U
}
println(emitVerilog(new MyOperators).substring(216))
class MyOperatorsTester(c: MyOperators) extends PeekPokeTester(c) {
  expect(c.io.out_add, 5)
  expect(c.io.out_sub, 1)
  expect(c.io.out_mul, 8)
}
assert(Driver(() => new MyOperators) {c => new MyOperatorsTester(c)})

In addition to addition, subtraction, and multplication, Chisel has mux and concatination operators. These are shown below. The `Mux` operates like traditional ternary operators, with the order (select, value if true, value if false). Note that `true.B` and `false.B` are the preferred ways to create Chisel Bool literals. The `Cat` ordering is MSB then LSB, and only takes two arguments. Concatinating more than two values requires multiple `Cat` calls or advanced Chisel and Scala features covered in later sections.

<span style="color:blue">**Example 2:**</span>

In [ ]:
class MyOperatorsTwo extends Module {
  val io = IO(new Bundle {
    val in      = Input(UInt(4.W))
    val out_mux = Output(UInt(4.W))
    val out_cat = Output(UInt(4.W))
  })

  val s = true.B
  io.out_mux := Mux(s, 3.U, 0.U) // should return 3.U, since s is true
  io.out_cat := Cat(2.U, 1.U) // concatenates 2 (10) with 1 (1) to give 5 (101)
}
println(emitVerilog(new MyOperatorsTwo).substring(216))
class MyOperatorsTwoTester(c: MyOperatorsTwo) extends PeekPokeTester(c) {
  expect(c.io.out_mux, 3)
  expect(c.io.out_cat, 5)
}
assert(Driver(() => new MyOperatorsTwo) {c => new MyOperatorsTwoTester(c)})

For a more complete list of Chisel operators, see the [Chisel cheatsheet](https://chisel.eecs.berkeley.edu/documentation.html). For the most complete list of operators and the implementation details, look through the [Chisel API](https://chisel.eecs.berkeley.edu/api/#package).

## Exercises

### <span style="color:blue">**Exercise 0 : MAC**</span>
Create a Chisel module that implements the multiply accumulate function, `(A*B)+C`, and passes the testbench.

In [ ]:
class MAC extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val in_c = Input(UInt(4.W))
    val out  = Output(UInt(8.W))
  })

  // YOUR CODE HERE
}
class MACTester(c: MAC) extends PeekPokeTester(c) {
  val cycles = 100
  import scala.util.Random
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    val in_c = Random.nextInt(16)
    poke(c.io.in_a, in_a)
    poke(c.io.in_b, in_b)
    poke(c.io.in_c, in_c)
    expect(c.io.out, in_a*in_b+in_c)
  }
}
assert(Driver(() => new MAC) {c => new MACTester(c)})

### <span style="color:blue">**Exercise 1 : Arbiter**</span>
The following circuit arbitrates data coming from a FIFO into two parallel processing units. The FIFO and processing elements (PEs) communicate with ready-valid interfaces. Construct the arbiter to send data to whichever PE is ready to receive data, prioritizing PE0 when both are ready to receive data. Remember that the arbiter should tell the FIFO that it's ready to receive data only when one of the PEs can receive data.

<img src="images/arbiter.png" width="687" height="177">

In [ ]:
class Arbiter extends Module {
  val io = IO(new Bundle {
    // FIFO
    val fifo_valid = Input(Bool())
    val fifo_ready = Output(Bool())
    val fifo_data  = Input(UInt(16.W))
    
    // PE0
    val pe0_valid  = Output(Bool())
    val pe0_ready  = Input(Bool())
    val pe0_data   = Output(UInt(16.W))
    
    // PE1
    val pe1_valid  = Output(Bool())
    val pe1_ready  = Input(Bool())
    val pe1_data   = Output(UInt(16.W))
  })

  // YOUR CODE HERE
}
class ArbiterTester(c: Arbiter) extends PeekPokeTester(c) {
  import scala.util.Random
  val data = Random.nextInt(65536)
  poke(c.io.fifo_data, data)
  
  for (i <- 0 until 8) {
    poke(c.io.fifo_valid, (i>>0)%2)
    poke(c.io.pe0_ready,  (i>>1)%2)
    poke(c.io.pe1_ready,  (i>>2)%2)

    expect(c.io.fifo_ready, i>1)
    expect(c.io.pe0_valid,  i==3 || i==7)
    expect(c.io.pe1_valid,  i==5)
    
    if (i == 3 || i ==7) {
      expect(c.io.pe0_data, data)
    } else if (i == 5) {
      expect(c.io.pe1_data, data)
    }
  }
}
assert(Driver(() => new Arbiter) {c => new ArbiterTester(c)})

### <span style="color:red">**Exercise 2 : Parameterized Adder (Optional)**</span>
This optional exercise exposes you to one of the most powerful features of Chisel, it's parameterization capabilities. To demonstrate this, we'll construct a parameterized adder that can either saturate the output when overflow occurs, or truncate the results (i.e. wrap around).

First, look at the `Module` below. The parameter we pass into it is called `saturate` and has type *Scala* `Boolean`. This is not a Chisel `Bool`. So, we're not creating a single hardware adder that can either saturate or truncate, but rather we're creating a *generator* that produces either a saturating hardware adder *or* a truncating hardware adder. The decision is made at compile time.

Next, notice the inputs and outputs are all 4-bit `UInt`s. Chisel has built-in width inferencing, and if you look at the [cheatsheet](https://chisel.eecs.berkeley.edu/doc/chisel-cheatsheet3.pdf), you'll see that the bitwidth of a normal summation is equal to the maximum bitwidth of the two inputs. This means that

```scala
val sum = io.in_a + io.in_b
```

will make `sum` a 4-bit wire, and the value will be the truncated result for 4-bit inputs. To check if the summation should saturate, you need to place the result in a 5-bit wire. This can be done with the `+&` summation, as seen on the cheatsheet.

```scala
val sum = io.in_a +& io.in_b
```

Finally, note that connecting a 4-bit `UInt` wire to a 5-bit `UInt` wire will truncate the MSB by default. You can use this to easily truncate the 5-bit sum for the non-saturating adder.

In [ ]:
class ParameterizedAdder(saturate: Boolean) extends Module {
  val io = IO(new Bundle {
    val in_a = Input(UInt(4.W))
    val in_b = Input(UInt(4.W))
    val out  = Output(UInt(4.W))
  })

  // YOUR CODE HERE
}
class ParameterizedAdderTester(c: ParameterizedAdder, saturate: Boolean) extends PeekPokeTester(c) {
  val cycles = 100
  import scala.util.Random
  import scala.math.min
  for (i <- 0 until cycles) {
    val in_a = Random.nextInt(16)
    val in_b = Random.nextInt(16)
    poke(c.io.in_a, in_a)
    poke(c.io.in_b, in_b)
    if (saturate) {
      expect(c.io.out, min(in_a+in_b, 15))
    } else {
      expect(c.io.out, (in_a+in_b)%16)
    }
  }
  
  // ensure we test saturation vs. truncation
  poke(c.io.in_a, 15)
  poke(c.io.in_b, 15)
  if (saturate) {
    expect(c.io.out, 15)
  } else {
    expect(c.io.out, 14)
  }
}
for (saturate <- Seq(true, false)) {
  assert(Driver(() => new ParameterizedAdder(saturate)) {c => new ParameterizedAdderTester(c, saturate)})
}